In [362]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

In [363]:
voyages = pd.read_csv('./voyages.csv')

In [364]:
#one-hot encoding for begin port and end port
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
X = voyages['begin_port_id'].unique()
X = X.reshape(-1, 1)
enc.fit(X)


y = voyages['end_port_id'].unique()
y = y.reshape(-1, 1)
enc.fit(y)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=True)

In [365]:
#build seq of begin ports, pad for nn
sequences_X = voyages.groupby('vessel').begin_port_id.apply(np.array).values
enc_sequences_X = [enc.transform(x.reshape(-1,1)).toarray() for x in sequences_X]
padded_enc_seqs_X = tf.keras.preprocessing.sequence.pad_sequences(enc_sequences_X)

In [366]:
#build seq of end ports
sequences_y = voyages.groupby('vessel').end_port_id.apply(np.array).values
enc_sequences_y = [enc.transform(x.reshape(-1,1)).toarray() for x in sequences_y]
padded_enc_seqs_y = tf.keras.preprocessing.sequence.pad_sequences(enc_sequences_y)

In [367]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.preprocessing.sequence import TimeseriesGenerator

X_train, X_test, y_train, y_test = train_test_split(padded_enc_seqs_X, padded_enc_seqs_y, test_size=0.33, random_state=66)

model = Sequential()
model.add(
    LSTM(
        units=1,
        return_sequences=True,
        activation='relu'
        )
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [368]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [369]:
model.fit(X_train, y_train, validation_data=(X_test, y_test))

4/4 [==============================] - 4s 269ms/step - loss: 0.0369 - accuracy: 0.9962 - val_loss: 0.0342 - val_accuracy: 0.9963


In [370]:
model(X_test)

<tf.Tensor: shape=(57, 41, 1), dtype=float32, numpy=
array([[[-0.00109608],
        [-0.00109608],
        [-0.00109608],
        ...,
        [ 0.01197573],
        [ 0.00825596],
        [ 0.00550033]],

       [[-0.00109608],
        [-0.00109608],
        [-0.00109608],
        ...,
        [ 0.00490368],
        [ 0.03201151],
        [ 0.02164739]],

       [[-0.00109608],
        [-0.00109608],
        [-0.00109608],
        ...,
        [ 0.02161765],
        [ 0.01538901],
        [ 0.0113741 ]],

       ...,

       [[-0.00109608],
        [-0.00109608],
        [-0.00109608],
        ...,
        [ 0.06553254],
        [ 0.0818387 ],
        [ 0.06598148]],

       [[-0.00109608],
        [-0.00109608],
        [-0.00109608],
        ...,
        [ 0.03691299],
        [ 0.02626995],
        [ 0.05777075]],

       [[-0.00109608],
        [-0.00109608],
        [-0.00109608],
        ...,
        [ 0.04250115],
        [ 0.03006133],
        [ 0.02348615]]], dtype=float32)>

In [373]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X)
print(le.transform(X))
print(le.inverse_transform(X))

[ 89  72  20  49  65  15   4  73   2  87  23  67  35   3  93  21  79  53
  19  51  57  58  62  22  37  81 110   1  10  38  16  69  99  83  59  61
  13  50  12  74  64   8 104  70  32  91  25  63  47  56  28  90  44 105
   5  17 101  60  36  76  52  85  88  30  24 116  82  97  29  78  34   9
  66 108  39 103  54  95 115  31  68 102   6  55  33  27  18  45  40  94
  14 117 107 113   7  75 111  41  48  46   0  86  92  26 112  77  98  84
  42 114 118 106  96  11  71 100 109  43  80]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored